In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Paths
DATA_DIR = "/content/drive/MyDrive/leaf-disease-toolbox/data/Pear/leaves"  # Base folder for images
CSV_PATH = "/content/drive/MyDrive/leaf-disease-toolbox/data/Pear/annotation/csv/diaMOSPlant.csv"

# Read CSV with semicolon delimiter
df = pd.read_csv(CSV_PATH, sep=";")
print("Original DataFrame shape:", df.shape)
print(df.head())

# Define columns
disease_cols = ['healthy', 'pear_slug', 'leaf_spot', 'curl']
severity_cols = ['severity_0', 'severity_1', 'severity_2', 'severity_3', 'severity_4']
all_label_cols = disease_cols + severity_cols

# Convert columns to numeric (rows with non-numeric severity values become NaN)
for col in disease_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
for col in severity_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with NaN in any severity column
df = df.dropna(subset=severity_cols).reset_index(drop=True)
print("Filtered DataFrame shape:", df.shape)

# Create a new "filepath" column based on the disease indicator.
folder_mapping = {
    "healthy": "healthy",
    "pear_slug": "slug",
    "leaf_spot": "spot",
    "curl": "curl"
}
def compute_filepath(row):
    for col in disease_cols:
        if row[col] == 1:
            return os.path.join(folder_mapping[col], row['filename'])
    return row['filename']
df['filepath'] = df.apply(compute_filepath, axis=1)
print("Example filepaths:")
print(df[['filename', 'filepath']].head(10))

# Split data (70% train, 15% validation, 15% test)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print("Train samples:", len(train_df), "Validation samples:", len(val_df), "Test samples:", len(test_df))

# Define a custom generator using flow_from_dataframe.
def multi_output_generator(datagen, dataframe, directory, batch_size, target_size, x_col, y_cols, disease_split=4, shuffle=True):
    base_gen = datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col=x_col,  # Use the "filepath" column
        y_col=y_cols,
        class_mode="raw",
        target_size=target_size,
        batch_size=batch_size,
        shuffle=shuffle
    )
    while True:
        images, labels = next(base_gen)
        disease_labels = labels[:, :disease_split]
        severity_labels = labels[:, disease_split:]
        yield images, {"disease": disease_labels, "severity": severity_labels}

# Generator parameters
BATCH_SIZE = 32
# We use (224,224) for most architectures; if you choose InceptionV3 you might need (299,299)
IMG_SIZE = (224, 224)

train_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_gen = multi_output_generator(train_datagen, train_df, DATA_DIR, BATCH_SIZE, IMG_SIZE,
                                   x_col="filepath", y_cols=all_label_cols, disease_split=len(disease_cols), shuffle=True)
val_gen = multi_output_generator(val_datagen, val_df, DATA_DIR, BATCH_SIZE, IMG_SIZE,
                                 x_col="filepath", y_cols=all_label_cols, disease_split=len(disease_cols), shuffle=False)
test_gen = multi_output_generator(test_datagen, test_df, DATA_DIR, BATCH_SIZE, IMG_SIZE,
                                  x_col="filepath", y_cols=all_label_cols, disease_split=len(disease_cols), shuffle=False)

train_steps = len(train_df) // BATCH_SIZE
val_steps   = len(val_df) // BATCH_SIZE
# For test, we use ceil so that all images are covered.
test_steps  = math.ceil(len(test_df) / BATCH_SIZE)
print("Steps per epoch: train =", train_steps, ", val =", val_steps, ", test =", test_steps)


Original DataFrame shape: (3005, 10)
  filename  healthy  pear_slug  leaf_spot  curl severity_0 severity_1  \
0   u2.jpg        0          0          1     0          0          1   
1   u4.jpg        0          0          1     0          0          1   
2   u6.jpg        0          0          1     0          0          1   
3   u8.jpg        0          0          1     0          0          0   
4  u10.jpg        0          0          1     0          0          1   

  severity_2 severity_3 severity_4  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          1          0          0  
4          0          0          0  
Filtered DataFrame shape: (2951, 10)
Example filepaths:
  filename      filepath
0   u2.jpg   spot/u2.jpg
1   u4.jpg   spot/u4.jpg
2   u6.jpg   spot/u6.jpg
3   u8.jpg   spot/u8.jpg
4  u10.jpg  spot/u10.jpg
5  u12.jpg  spot/u12.jpg
6  u14.jpg  spot/u14.jpg
7  u16.jpg  spot/u16.jpg
8  u18.jpg  spot/u18.

In [4]:
import tensorflow as tf

def get_conv_base(architecture, input_shape):
    arch = architecture.lower()
    if arch == 'vgg16':
        return tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif arch == 'vgg19':
        return tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    elif arch == 'resnet50':
        return tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif arch == 'inceptionv3':
        return tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    elif arch == 'mobilenetv2':
        return tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif arch == 'efficientnetb0':
        return tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unknown architecture: " + architecture)

def build_full_model(architecture, input_shape=(224,224,3), fine_tune=False, fine_tune_at=None):
    base_model = get_conv_base(architecture, input_shape)
    # Initially, freeze the base model.
    if not fine_tune:
        base_model.trainable = False
    else:
        # If fine tuning, unfreeze layers from index fine_tune_at onward.
        if fine_tune_at is not None:
            for layer in base_model.layers[:fine_tune_at]:
                layer.trainable = False
            for layer in base_model.layers[fine_tune_at:]:
                layer.trainable = True
        else:
            base_model.trainable = True  # unfreeze all
    inputs = tf.keras.Input(shape=input_shape)
    # Pass inputs through the base model.
    x = base_model(inputs, training=not fine_tune)  # use training mode if fine tuning
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)

    # Disease branch (4 classes)
    x1 = tf.keras.layers.Dense(1024, activation='relu')(x)
    x1 = tf.keras.layers.Dropout(0.5)(x1)
    disease_output = tf.keras.layers.Dense(4, activation='softmax', name='disease')(x1)

    # Severity branch (5 classes)
    x2 = tf.keras.layers.Dense(1024, activation='relu')(x)
    x2 = tf.keras.layers.Dropout(0.5)(x2)
    severity_output = tf.keras.layers.Dense(5, activation='softmax', name='severity')(x2)

    model = tf.keras.Model(inputs=inputs, outputs=[disease_output, severity_output])
    return model


In [ ]:
architectures = ['vgg16', 'resnet50', 'mobilenetv2', 'efficientnetb0']
initial_epochs = 5
fine_tune_epochs = 5

results = {}

for arch in architectures:
    print("\n===================================")
    print("Training architecture:", arch)
    print("===================================")
    # Build and compile the model with frozen base
    model = build_full_model(arch, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), fine_tune=False)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5, momentum=0.9),
                  loss={"disease": "categorical_crossentropy", "severity": "categorical_crossentropy"},
                  metrics={"disease": "accuracy", "severity": "accuracy"})
    model.summary()

    history = model.fit(train_gen,
                        steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=initial_epochs)

    # Now fine-tune the model: unfreeze the top 30% of the base model layers.
    # We assume that the base model is the first layer in our model's layers list.
    base_model = model.layers[1]  # because our model was built as Input -> base_model -> ...
    num_layers = len(base_model.layers)
    fine_tune_at = int(num_layers * 0.7)  # freeze first 70%, unfreeze remaining 30%
    print(f"Fine-tuning: freezing first {fine_tune_at} layers out of {num_layers}.")
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True

    # Recompile the model with a lower learning rate for fine tuning.
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5, momentum=0.9),
                  loss={"disease": "categorical_crossentropy", "severity": "categorical_crossentropy"},
                  metrics={"disease": "accuracy", "severity": "accuracy"})

    history_ft = model.fit(train_gen,
                           steps_per_epoch=train_steps,
                           validation_data=val_gen,
                           validation_steps=val_steps,
                           epochs=fine_tune_epochs)

    # Evaluate the model on the test set.
    # Note: Because test_steps was computed with ceil, we clip predictions to the actual number of test samples.
    preds = model.predict(test_gen, steps=test_steps, verbose=1)
    num_test = len(test_df)
    pred_disease = preds[0][:num_test].argmax(axis=1)
    pred_severity = preds[1][:num_test].argmax(axis=1)

    true_disease = test_df[disease_cols].values.argmax(axis=1)
    true_severity = test_df[severity_cols].values.argmax(axis=1)

    disease_correct = np.sum(true_disease == pred_disease)
    severity_correct = np.sum(true_severity == pred_severity)
    disease_acc = disease_correct / len(true_disease)
    severity_acc = severity_correct / len(true_severity)

    print(f"\n[{arch}] Disease classification: {disease_correct} / {len(true_disease)} correct ({disease_acc*100:.2f}%)")
    print(f"[{arch}] Severity classification: {severity_correct} / {len(true_severity)} correct ({severity_acc*100:.2f}%)")

    results[arch] = {"disease_accuracy": disease_acc, "severity_accuracy": severity_acc}

# Print summary of results
print("\n=== Summary of Test Accuracies ===")
for arch, acc in results.items():
    print(f"{arch}: Disease = {acc['disease_accuracy']*100:.2f}%, Severity = {acc['severity_accuracy']*100:.2f}%")



Training architecture: vgg16
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ vgg16 (Functional)        │ (None, 7, 7, 512)      │     14,714,688 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ avg_pool                  │ (None, 512)            │              0 │ vgg16[0][0]            │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 512)            │          2,048 │ avg_pool[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 512)            │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1024)           │        525,312 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1024)           │        525,312 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1024)           │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 1024)           │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ disease (Dense)           │ (None, 4)              │          4,100 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ severity (Dense)          │ (None, 5)              │          5,125 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 15,776,585 (60.18 MB)

 Trainable params: 1,060,873 (4.05 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mobilenetv2_1.00_224      │ (None, 7, 7, 1280)     │      2,257,984 │ input_layer_5[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ avg_pool                  │ (None, 1280)           │              0 │ mobilenetv2_1.00_224[… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 1280)           │          5,120 │ avg_pool[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 1280)           │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 1024)           │      1,311,744 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 1024)           │      1,311,744 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 1024)           │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 1024)           │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ disease (Dense)           │ (None, 4)              │          4,100 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ severity (Dense)          │ (None, 5)              │          5,125 │ dropout_5[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,895,817 (18.68 MB)

 Trainable params: 2,635,273 (10.05 MB)

 Non-trainable params: 2,260,544 (8.62 MB)

Epoch 1/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - disease_accuracy: 0.5509 - disease_loss: 1.1635 - loss: 3.0811 - severity_accuracy: 0.3496 - severity_loss: 1.9183 - val_disease_accuracy: 0.7372 - val_disease_loss: 0.5604 - val_loss: 1.6371 - val_severity_accuracy: 0.4988 - val_severity_loss: 1.0747
Epoch 2/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 269s 4s/step - disease_accuracy: 0.8177 - disease_loss: 0.4537 - loss: 1.6406 - severity_accuracy: 0.5336 - severity_loss: 1.1870 - val_disease_accuracy: 0.8054 - val_disease_loss: 0.4056 - val_loss: 1.3866 - val_severity_accuracy: 0.5813 - val_severity_loss: 0.9774
Epoch 3/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 245s 4s/step - disease_accuracy: 0.8496 - disease_loss: 0.3531 - loss: 1.3946 - severity_accuracy: 0.5644 - severity_loss: 1.0416 - val_disease_accuracy: 0.8473 - val_disease_loss: 0.3721 - val_loss: 1.2982 - val_severity_accuracy: 0.6059 - val_severity_loss: 0.9233
Epoch 4/5
64/64 ━━━━━━━━━━━━━━━━━━━━ 256s 4s/step - disease_accuracy: 0.8391 - disease_l